In [1]:
%autosave 0
%matplotlib inline

import os, sys
sys.path.insert(0, os.path.expanduser('~/git/github/pymc-devs/pymc3'))

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc
import daft
import pymc3 as pm
import numpy as np
import seaborn as sns

from IPython.display import display
from IPython.display import HTML
import IPython.core.display as display

Autosave disabled


In [1]:
%%HTML
<style>
img[src$="centerme"] {
  display:block;
  margin: 0 auto;
}
</style>

<br />

<div style="text-align: center;">
<font size="7"><b>Variational inference in </b></font>
<div style="display: inline-block; vertical-align: middle;">
<img src="./pymc3_logo.jpg" width=200 height=200 /></div> 
</div>
<br />
<br />
<br />
<br />
<div style="text-align: right;">
<font size="6">Taku Yoshioka</font>
</div>

<br />

# PyMC3
* Python library for Bayesian inference

    * Define probabilistic models consisting of random variables (RVs)
    * Draw samples of RVs from the posterior distribution with MCMC
    * __Fit parametrized approximate posterior to data with variational inference__

# Agenda
* Bayesian inference and variational inference
* Stochastic gradient for optimization
* Example: Hierarchical linear regression and Bayesian neural network
* Autoencoding Variational Bayes (AEVB)
* Example: Convolutional variational autoencoder

# Probabilistic model
* Decomposition of the joint probability of all random variables (RVs)

$$
p(\mathbf{x},\mathbf{z}) = p(\mathbf{x}|\mathbf{z})p(\mathbf{z})
$$
* $p(\mathbf{x},\mathbf{z})$: joint distribution of $\mathbf{x}$ and $\mathbf{z}$
* $p(\mathbf{z})$: prior distribution on $\mathbf{z}$
* $p(\mathbf{x}|\mathbf{z})$: likelihood of data $\mathbf{x}$ given $\mathbf{z}$

# Bayesian inference
* Infer the posterior distribution $p(\mathbf{z}|\mathbf{x})$, the distribution of the unkown RVs $\mathbf{z}$ given observations $\mathbf{x}$ (which is also RVs), based on the Bayes theorem:

$$
p(\mathbf{z}|\mathbf{x}) = \frac{p(\mathbf{x},\mathbf{z})}{p(\mathbf{x})}=\frac{p(\mathbf{x}|\mathbf{z})p(\mathbf{z})}{p(\mathbf{x})}
$$

# Variational inference
* Approximate $p(\mathbf{z}|\mathbf{x})$ by variational distribution $q(\mathbf{z})$
* Maximize evidence lower bound (ELBO) ${\cal L}(q)$ w.r.t. $q$ minimizes $KL[q(\mathbf{z})||p(\mathbf{z}|\mathbf{x})]$

\begin{eqnarray}
\cal{L}(q) & = & \mathbb{E}_{q(\mathbf{z})}\left[\log p(\mathbf{x},\mathbf{z}) - \log q(\mathbf{z})\right] \\
           & = & \mathbb{E}_{q(\mathbf{z})}\left[\log p(\mathbf{x}|\mathbf{z})\right] - KL\left[\log q(\mathbf{z})||p(\mathbf{z})\right] \\
           & = & \log p(\mathbf{x}) + KL[q(\mathbf{z})||p(\mathbf{z}|\mathbf{x})]
\end{eqnarray}

* Pros: Can obtain tractable solution under factorized $q(\mathbf{x})$ and conjugate prior
* Cons: Restricted probabilistic models because of the tractability

# Meanfield approximation
* Normal distribution as $q(\mathbf{z})$ with parameters $\gamma$ (variational parameters)

$$
q_{\lambda}(\mathbf{z}) = \prod_{d=1}^{D}N(z_{d};\mu_{d},\sigma_{d}^{2}), \gamma=\left\{\mu_{d}, \sigma_{d}\right\}_{d=1}^{D}
$$

* Maximize ELBO wrt $\gamma$ with stochastic gradient

# Reparametrization
* Reparametrization $\mathbf{z} = \mathbf{\sigma}\odot\mathbf{\epsilon}+\mathbf{\mu}$ makes the calculation of gradients easy
    
$$
\begin{eqnarray}
\mathbb{E}_{q}\left[\log p(\mathbf{x},\mathbf{z})\right] & = & \int p(\mathbf{\epsilon})\log p(\mathbf{x},\mathbf{\sigma}\odot\mathbf{\epsilon}+\mathbf{\mu}) d\mathbf{\epsilon}, \\
\partial_{\gamma}\mathbb{E}_{q}\left[\log p(\mathbf{x},\mathbf{z})\right] & = & \int p(\mathbf{\epsilon})\partial_{\gamma}\log p(\mathbf{x},\mathbf{\sigma}\odot\mathbf{\epsilon}+\mathbf{\mu}) d\mathbf{\epsilon}
\end{eqnarray}
$$

* Monte Carlo sampling of the gradient (its expectation it the true gradient)

# Three steps for VI in PyMC3
1. Define model

2. Run optimization

3. Draw samples from variational posterior

# Example: Hierarchical regression
<div style="text-align: center;">
<div style="display: inline-block; vertical-align: middle;">
<img src="./hierarchical_regression.png" width=500 height=500 /></div> 
</div>

\begin{eqnarray}
\alpha_{c} &\sim & {\cal N}(\mu_{\alpha}, \sigma_{\alpha}^{2}) \\
\beta_{c} &\sim & {\cal N}(\mu_{\beta}, \sigma_{\beta}^{2}) \\
radon_{i,c} & = & \alpha_{c} + \beta_{c} * floor_{i,c} + \epsilon_{c}
\end{eqnarray}

# 1. Define model

In [ ]:
with pm.Model() as hierarchical_model:
    mu_a = pm.Normal('mu_alpha', mu=0., sd=100**2)
    sigma_a = pm.Uniform('sigma_alpha', lower=0, upper=100)
    mu_b = pm.Normal('mu_beta', mu=0., sd=100**2)
    sigma_b = pm.Uniform('sigma_beta', lower=0, upper=100)
    
    a = pm.Normal('alpha', mu=mu_a, sd=sigma_a, shape=n_counties)
    b = pm.Normal('beta', mu=mu_b, sd=sigma_b, shape=n_counties)
    
    eps = pm.Uniform('eps', lower=0, upper=100)
    
    radon_est = a[county_idx] + b[county_idx] * data.floor.values
    
    radon_like = pm.Normal('radon_like', mu=radon_est, sd=eps, observed=data.log_radon)
    

# 2. Run optimization

In [ ]:
with hierarchical_model:
    v_params = pm.variational.advi(n=100000)
    

# 3. Draw samples

In [ ]:
with hierarchical_model:
    trace = pm.variational.sample_vp(v_params, draws=5000)
    

# Example: Bayesian neural network
http://twiecki.github.io/blog/2016/06/01/bayesian-deep-learning/

\begin{eqnarray}
\mathbf{W}_{0/1/2} & \sim & {\cal N}(0, 1) \\
\mathbf{h}_{0} & = & \mathbf{x} \\
\mathbf{h}_{i} & = & {\rm sigm}(\mathbf{W}_{i-1}\cdot\mathbf{h}_{i-1}) \\
\mathbf{y} & \sim & {\cal N}({\rm sigm}(\mathbf{W}_{2}\cdot\mathbf{h}_{2}))
\end{eqnarray}

In [ ]:
with pm.Model() as neural_network:
    # Weights from input to hidden layer
    weights_in_1 = pm.Normal('w_in_1', 0, sd=1, shape=(X.shape[1], n_hidden), 
                             testval=init_1)
    
    # Weights from 1st to 2nd layer
    weights_1_2 = pm.Normal('w_1_2', 0, sd=1, shape=(n_hidden, n_hidden), 
                            testval=init_2)
    
    # Weights from hidden layer to output
    weights_2_out = pm.Normal('w_2_out', 0, sd=1, shape=(n_hidden,), 
                              testval=init_out)
    
    # Build neural-network using tanh activation function
    act_1 = T.tanh(T.dot(ann_input, weights_in_1))
    act_2 = T.tanh(T.dot(act_1, weights_1_2))
    act_out = T.nnet.sigmoid(T.dot(act_2, weights_2_out))
    
    # Binary classification -> Bernoulli likelihood
    out = pm.Bernoulli('out', act_out, observed=ann_output)


# Latent variables for observations
* Consider the case where each of i.i.d. observations latent variables

$$
\log p(\mathbf{X},\mathbf{Z},\Theta) = \sum_{i=1}^{N}\log p(\mathbf{x}_{i},\mathbf{z}_{i},\Theta)
$$

* Two types of latent variables

    * $\mathbf{z}_{i}$: Local random variables, related to each sample
    * $\Theta$: Global random variables, shared with all samples

# Autoencoding variational Bayes (1/2)
* Approximate $p(\Theta,\mathbf{Z}|\mathbf{X})$ by $q(\Theta)\prod_{i=1}^{N}q(\mathbf{z}_{i})$
* Assignment of variational parameters for each $\mathbf{z}_{i}$ does not generalize to new data
* Estimate the parameters of $q(\mathbf{z}_{i})$ (means and stds) by neural network with parameters $\nu$
* Log likelihood $p(\mathbf{x}_{i}|\mathbf{z}_{i},\Theta)$ can also parametrized ($\eta$)

# Autoencoding variational Bayes (2/2)
* Optimize ELBO wrt $\gamma$, $\nu$, and $\eta$

\begin{eqnarray}
{\cal L}(\gamma,\nu,\eta) & = & 
    \mathbf{c}_{o}\mathbb{E}_{q(\Theta)}\left[
        \sum_{i=1}^{N}\mathbb{E}_{q(\mathbf{z}_{i})}\left[
            \log p(\mathbf{y}_{i}|\mathbf{z}_{i},\Theta,\eta)
        \right]
    \right] \\
    && - \mathbf{c}_{g}KL\left[q(\Theta)||p(\Theta)\right]
       - \mathbf{c}_{l}\sum_{i=1}^{N}KL\left[q(\mathbf{z}_{i})||p(\mathbf{z}_{i})\right]
\end{eqnarray}

* $\mathbf{c}_{g/l/o}$ weighting constants
* $\mathbf{c}_{l/o}=N/M$ for mini-batches with size $M$

# Example: Convolutional variational autoencoder
https://taku-y.github.io/notebook/20161105/convolutional_vae_keras_advi.html

$$
\begin{eqnarray}
\tilde{\mathbf{x}} & \sim & {\cal N}(f(g(\mathbf{x})), 1)
\end{eqnarray}
$$

In [ ]:
with pm.Model() as model:
    # Hidden variables
    zs = pm.Normal('zs', mu=0, sd=1, shape=(minibatch_size, dim_hidden), dtype='float32')

    # Decoder and its parameters
    dec = Decoder(zs, net=cnn_dec)

    # Observation model
    xs_ = pm.Normal('xs_', mu=dec.out.ravel(), sd=1, observed=xs_t.ravel(), dtype='float32')


# Encoder

In [ ]:
def cnn_enc(xs, latent_dim, nb_filters=64, nb_conv=3, intermediate_dim=128):
    input_layer = InputLayer(input_tensor=xs,
                             batch_input_shape=xs.get_value().shape)
    model = Sequential()
    model.add(input_layer)

    cp1 = {'border_mode': 'same', 'activation': 'relu'}
    cp2 = {'border_mode': 'same', 'activation': 'relu', 'subsample': (2, 2)}
    cp3 = {'border_mode': 'same', 'activation': 'relu', 'subsample': (1, 1)}
    cp4 = cp3

    model.add(Convolution2D(1, 2, 2, **cp1))
    model.add(Convolution2D(nb_filters, 2, 2, **cp2))
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv, **cp3))
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv, **cp4))
    model.add(Flatten())
    model.add(Dense(intermediate_dim, activation='relu'))
    model.add(Dense(2 * latent_dim))

    return model

# Limitation
* Cannot handle discrete variables

    * Marginalization of discrete variables (Gaussian mixture, latent dirichlet allocation)

* Limited flexibility of the posterior approximation because of the mean-field approximation

# Future work
* Normalizing flows ([#1490](https://github.com/pymc-devs/pymc3/pull/1490))
* Stein variational gradient descent ([#1549](https://github.com/pymc-devs/pymc3/pull/1549))